In [13]:
#pip install gym
#pip install pygame

In [14]:
import gym
import numpy as np

In [17]:
env = gym.make("Taxi-v3", render_mode="rgb_array")
state = env.reset()

In [18]:
print(state)
#env.render()

(68, {'prob': 1.0, 'action_mask': array([1, 0, 1, 1, 0, 0], dtype=int8)})


array([[[110, 109, 106],
        [110, 109, 106],
        [124, 122, 122],
        ...,
        [108, 111, 109],
        [108, 111, 109],
        [118, 119, 119]],

       [[110, 109, 106],
        [110, 109, 106],
        [124, 122, 122],
        ...,
        [108, 111, 109],
        [108, 111, 109],
        [118, 119, 119]],

       [[114, 116, 115],
        [114, 116, 115],
        [126, 127, 126],
        ...,
        [112, 113, 111],
        [112, 113, 111],
        [118, 117, 115]],

       ...,

       [[116, 115, 116],
        [116, 115, 116],
        [106, 107, 108],
        ...,
        [113, 115, 114],
        [113, 115, 114],
        [117, 114, 117]],

       [[116, 115, 116],
        [116, 115, 116],
        [106, 107, 108],
        ...,
        [113, 115, 114],
        [113, 115, 114],
        [117, 114, 117]],

       [[115, 112, 112],
        [115, 112, 112],
        [119, 119, 117],
        ...,
        [123, 119, 118],
        [123, 119, 118],
        [114, 114, 117]]

# Possible Actions

In [19]:
n_states = env.observation_space.n
n_actions = env.action_space.n

In [31]:
print(env.step(env.action_space.sample()))

(227, -1, False, False, {'prob': 1.0, 'action_mask': array([1, 1, 1, 1, 0, 0], dtype=int8)})


In [44]:
print(env.reset())

(341, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})


In [64]:
print(env.action_space)

Discrete(6)


# Random Actions

In [32]:
state = env.reset()
counter = 0
g = 0
reward = None
while reward != 20:
    state, reward, done, _bool, info = env.step(env.action_space.sample())
    counter += 1
    g += reward
print("Solved in {} Steps with a total reward of {}".format(counter, g))

Solved in 1240 Steps with a total reward of -4738


# Reinforced Learning

In [55]:
episodes = 5

In [46]:
Q = np.zeros([n_states, n_actions])
Q.shape

(500, 6)

In [47]:
#To keep history of Q values
Q_hist = np.zeros([n_states, n_actions])
Q_hist.shape

(500, 6)

In [48]:
alpha = 0.618
G = 0

In [49]:
print(env.reset()[0])

21


In [56]:
for episode in range(1, episodes+1):
    done = False
    G, reward = 0, 0
    state_info = env.reset()
    firstState = state_info
    print("Initial State = {}".format(firstState))
    
    while reward != 20:
        action = np.argmax(Q[state_info[0]])
        next_state, reward, done, _bool, info = env.step(action)
        Q[state_info[0], action] += alpha * (reward + np.max(Q[next_state]) - Q[state_info[0], action])
        G += reward
        state_info = (next_state, info)
        
        #To keep a history a Q values in a multi dimensional array
        Q_hist = np.dstack(Q)
finalState = state_info
print("Final State = {}".format(finalState))

Initial State = (41, {'prob': 1.0, 'action_mask': array([1, 0, 1, 0, 0, 0], dtype=int8)})
Initial State = (13, {'prob': 1.0, 'action_mask': array([1, 0, 1, 0, 0, 0], dtype=int8)})
Initial State = (33, {'prob': 1.0, 'action_mask': array([1, 0, 0, 1, 0, 0], dtype=int8)})
Initial State = (327, {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})
Initial State = (103, {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})
Final State = (475, {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 1, 0], dtype=int8)})


In [66]:
#To render agent choices for optimal Q
state = env.reset()
done = None
while done != True:
    action = np.argmax(Q[state])
    state, reward, done, info = env.step(action)
    env.render()


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Let's add reward discouting and a decaying epsilon for randomness

In [65]:
epsilon = 1
gamma = 0.95

rewardTracker = []
Q = np.zeros([n_states, n_states])
episodes = 5000
G = 0
alpha = 0.618

for episode in range(1, episodes+1):
    done = False
    G, reward = 0, 0
    state = env.reset()
    
    while done != True:
        
        if np.random.rand() > epsilon:
            action = np.argmax(Q[state[0]])
        else:
            action = env.action_space.sample()
            epsilon -= 10**3
            
        if action == 6:
            action = 0
            
        state2, reward, done, _bool, info = env.step(action)
        Q[state[0], action] += alpha * (reward + gamma * np.max(Q[state2]) - Q[state[0], action])
        G += reward
        state = (state2, info)
    
    rewardTracker.append(G)
        
average_end = sum(rewardTracker[episodes-100:episodes]) / 100.0
if average_end > 70:
    print("----------------------------")
    print("Solved after {} episodes with average return of {}".format(episodes-100, average_end))

print("Average return of {}".format(sum(rewardTracker)/len(rewardTracker)))

KeyboardInterrupt: 

In [ ]:
def e_greedy(eps, Q, state, episode):
    if np.random.rand() > eps:
        action = np.argmax(Q[state, :] + np.random.randn(1, n_actions)/(episode/4))
    else:
        action = env.action_space.sample()
        eps -= 10**-5
        
    return action, eps

In [ ]:
#Not yet adapted to formal case for this python version
#To define optimal hyper parameters
def learn_Q(alpha, gamma, eps, numTrainingEpisodes, numTrainingSteps):
    global Q_star
    Q = np.zeros([env.observation_space.n, env.action_space.n])
    rewardTracker = []
    
    for episode in range(1, numTrainingEpisodes+1):
        G = 0
        state = env.reset()
        
        for step in range(1, numTrainingSteps):
            
            action, eps = e_greedy(eps, Q, state, episode)
            state2, reward, done, info = env.step(action)
            Q[state, action] += alpha*( reward + gamme * np.max(Q[state2]) - Q[state, action])
            state = state2
            G += reward
            
        rewardTracker.append(Q)
    
        if (episode % (numTrainingEpisodes*.10) == 0 and episode != 0):
            print("Alpha {} Gamma {} Epsilon {:04.3f} Episode {} of {}".format(alpha, gamma, eps, episode, numTrainingEpisode))
            print("Average Total Return: {}".format(sum(rewardTracker)/episode))
            
        average_end = sum(rewardTracker[episode-100:episode]) / 100.0
        if average_end > 70:
            print("----------------------------")
            print("Solved after {} episodes with average return of {}".format(episode-100, average_end))
            Q_star = Q
            break
    Q_start = Q